## <center> Predict Sentiment Analysis over Movie Reviews

In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings('ignore')

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
#Import dataset
df_train = pd.read_csv('data/labeledTrainData.tsv', delimiter='\t')

df_train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


### Function to clean text.

In [3]:
#Function to clean text
def clean_text(input_text):
    """
    Processes the give text and removes all non words, digits, single letters and extra spaces.

    Parameters
    -----------
    1. input_text = Text to clean.
    2. token = 'word' or 'sentence'

    Returns: Text.

    """

    text = re.sub(r'\W',' ', input_text) #Remove all non words
    text = re.sub(r'\d+',' ', text) #Remove all digits
    text = text.lower() #Converting text into lowercase
    text = re.sub(r'\s+[a-z]\s+',' ', text) #Remove all single letters
    text = re.sub(r'^\s+','', text) #Remove space from start of text
    text = re.sub(r'\s+$','', text) #Remove space from end of text
    text = re.sub(r'\s+',' ', text) #Remove all multi space    
    text = text.split(' ') #Split the words into tokens
    text = [word for word in text if word not in stop_words] #Remove stopwords
    text = [WordNetLemmatizer().lemmatize(word) for word in text] #Lemmatize the words(get root form)
    text = ' '.join(text)

    return text

### Validate the clean text function

In [4]:
#Actual Text
temp = df_train.loc[0,'review']
temp

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [5]:
#Cleaned text
clean_text(temp)

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle message mj feeling towards press also obvious message drug bad kay br br visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice br br actual feature film bit finally start minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pesci character ranted wanted people know supplying drug etc dunno maybe hate mj music br br lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually director hate wor

In [6]:
#Apply clean text over movie reviews
df_train['processed_reviews'] = df_train['review'].apply(lambda x: clean_text(x))

In [7]:
#Review dataset
df_train.head()

,id,sentiment,review,processed_reviews
0,5814_8,1,With all this stuff going down at the moment w...,stuff going moment mj started listening music ...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertaining f...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell giving welcom...
3,3630_4,0,It must be assumed that those who praised this...,must assumed praised film greatest filmed oper...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


In [8]:
#Finding average total length of words in review
df_train['processed_reviews'].apply(lambda x: len(x.split(' '))).mean()

123.35284

## Preprocessing the data

In [9]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train['processed_reviews'])
list_tokens = tokenizer.texts_to_sequences(df_train['processed_reviews'])

In [10]:
maxlen = 130 #Selected from mean of text length
X_train = pad_sequences(list_tokens, maxlen=maxlen)
y_train = df_train['sentiment']

## Building Model

In [11]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
batch_size = 100
epochs = 3
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 32s 2ms/step - loss: 0.4583 - accuracy: 0.7844 - val_loss: 0.3230 - val_accuracy: 0.8634
Epoch 2/3
20000/20000 [==============================] - 32s 2ms/step - loss: 0.2466 - accuracy: 0.9047 - val_loss: 0.3253 - val_accuracy: 0.8640
Epoch 3/3
20000/20000 [==============================] - 31s 2ms/step - loss: 0.1831 - accuracy: 0.9327 - val_loss: 0.3453 - val_accuracy: 0.8644


## Read the Test Data

In [13]:
df_test = pd.read_csv('data/testData.tsv', delimiter='\t')
df_test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


### Clean Test data

In [14]:
df_test['processed_review'] = df_test['review'].apply(lambda x: clean_text(x))

### Preprocess Test Data

In [15]:
list_sentences_test = df_test['processed_review']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

### Make Prediction

In [16]:
prediction = model.predict(X_test)

In [17]:
#Set the prediction value
p=[]
for val in prediction:
    if val > 0.5:
        p.append(1)
    else:
        p.append(0)
        
sr_pred = pd.Series(data=p, name='sentiment')
sr_pred[:5]

0    1
1    0
2    1
3    1
4    1
Name: sentiment, dtype: int64

### Preparing Submission File

In [18]:
submission = pd.DataFrame(columns=['id', 'sentiment'])
submission['id'] = df_test['id']
submission['sentiment'] = sr_pred
submission.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,1


In [19]:
submission.to_csv('submission/first_submission.csv', index=False)

### Secured score - 0.85456